In [2]:
from queue import LifoQueue
#Turing our bfs and dfs into goal based agents:
class GoalBasedAgent:
  def __init__(self,goal):
    self.goal=goal
  def formulate_goal(self,percept):
    if percept==self.goal:
      return True
    else:
      return False
  def act(self,percept,environment):
    if self.formulate_goal(percept)==True:
      print("Goal Reached")
    else:
      print("Searching")
      environment.dfs(percept,self.goal)

class Environment:
  def __init__(self, graph):
    self.graph = graph
  def get_percept(self, node):
    return node
  def dfs(self,start,goal):
  #//we will have a frontier that stores the nodes to be explored
  #//also have a list of nodes that already have been explored
    frontier=LifoQueue(maxsize=0)
    explored=[]
    frontier.put(start)
    explored.append(start)
    while (frontier.empty())==False:
      node=frontier.get()
      print(node," ")
      if node==goal:
       break
      for child in self.graph.get(node,[]) :
        if child not in explored:
         frontier.put(child)
         explored.append(child)


def runAgent(agent,environment,start) :
  percept=environment.get_percept(start)
  action=agent.act(percept,environment)
  print(action)

# Tree Representation
tree = {
'A': ['B', 'C'],
'B': ['D', 'E'],
'C': ['F', 'G'],
'D': ['H'],
'E': [],
'F': ['I'],
'G': [],
'H': [],
'I': []
}
# Define Start and Goal Nodes
start_node = 'A'
goal_node = 'I'
# Create instances of agent and environment
agent = GoalBasedAgent(goal_node)
environment = Environment(tree)
# Run the agent
runAgent(agent, environment, start_node)

Searching
A  
C  
G  
F  
I  
None


In [3]:
#Turing our  dfs into depth limited search:
class GoalBasedAgent:
  def __init__(self,goal,limit):
    self.goal=goal
    self.limit=limit
  def formulate_goal(self,percept):
    if percept==self.goal:
      return True
    else:
      return False
  def act(self,percept,environment):
    if self.formulate_goal(percept)==True:
      print("Goal Reached")
      return

    else:
      print("Searching")
      environment.dls(percept,self.goal,self.limit)

class Environment:
  def __init__(self, graph,limit):
    self.graph = graph
    self.limit=limit
  def get_percept(self, node):
    return node
  def dls(self,start,goal,limit):
  #//we will have a frontier that stores the nodes to be explored
  #//also have a list of nodes that already have been explored
  #we need to track the depth of each node individually and pass it along during exploration.
  # We can achieve this by storing nodes and their depths as tuples in the
    # frontier.
    frontier=LifoQueue(maxsize=0)
    explored=[]
    frontier.put((start,0))

    explored.append(start)
    while (frontier.empty())==False:
      node,depth=frontier.get()
      print(node," ")
      if node==goal:
       return True
      else:
        if depth>=self.limit:
          continue
        for child in self.graph.get(node,[]) :
          if child not in explored:
            frontier.put((child,depth+1))

            explored.append(child)
    return False

def runAgent(agent,environment,start) :
  percept=environment.get_percept(start)
  action=agent.act(percept,environment)
  print(action)

# Tree Representation
tree = {
'A': ['B', 'C'],
'B': ['D', 'E'],
'C': ['F', 'G'],
'D': ['H'],
'E': [],
'F': ['I'],
'G': [],
'H': [],
'I': []
}
# Define Start and Goal Nodes
start_node = 'A'
goal_node = 'I'
# Create instances of agent and environment
agent = GoalBasedAgent(goal_node,3)
environment = Environment(tree,3)
# Run the agent
runAgent(agent, environment, start_node)

Searching
A  
C  
G  
F  
I  
None


In [5]:
class UtilityBasedAgent:
  def __init__(self,goal):
    self.goal=goal
  def formulate_goal(self,percept):
    if percept==self.goal:
      return True
    else:
      return False
  def act(self,percept,environment):
    if self.formulate_goal(percept)==True:
      print("Goal Reached")
    else:
      print("Searching")
      environment.ucs(percept,self.goal)

class Environment:
  def __init__(self, graph):
    self.graph = graph

  def get_percept(self, node):
    return node
  def ucs(self,start,goal):
    frontier=[]
    explored =set()
    frontier.append((start,0))
    cost_so_far={start:0}
    came_from={start:None}


    while frontier:
      frontier.sort(key=lambda x:x[1])
      node,cost=frontier.pop(0)

      if node in explored:
        continue
      explored.add(node)
      if node==goal:
        path=[]
        while node is not None:
          path.append(node)
          node=came_from[node]
        path.reverse()
        print(path)
        print("Cost: ",cost)
        return True

      for child,Cost in self.graph[node].items():
        if child not in explored:
          newCost=cost+Cost
          if child not in cost_so_far or newCost<cost_so_far[child]:# WE DO THIS LIKE IN DIJISKSTRA WE ONLY UPDTAE IF PATH COST IS LOWER
              frontier.append((child,newCost))

              cost_so_far[child]=newCost
              came_from[child]=node
    return False

def runAgent(agent,environment,start) :
  percept=environment.get_percept(start)
  action=agent.act(percept,environment)
  print(action)

# Tree Representation

graph = {
'A': {'B': 2, 'C': 1},
'B': {'D': 4, 'E': 3},
'C': {'F': 1, 'G': 5},
'D': {'H': 2},
'E': {},
'F': {'I': 6},
'G': {},
'H': {},
'I': {}
}
# Define Start and Goal Nodes
start_node = 'A'
goal_node = 'I'
# Create instances of agent and environment
agent = UtilityBasedAgent(goal_node)
environment = Environment(graph)
# Run the agent
runAgent(agent, environment, start_node)

Searching
['A', 'C', 'F', 'I']
Cost:  8
None


In [7]:
from collections import deque

def bidirectional_search(graph, start, goal):
    if start == goal:
        return [start]

    # Initialize frontiers for forward and backward searches
    forward_queue = deque([start])
    backward_queue = deque([goal])

    # Track visited nodes and their parents for path reconstruction
    forward_visited = {start: None}
    backward_visited = {goal: None}

    while forward_queue and backward_queue:
        # Forward search step
        meeting_point = expand_frontier(graph, forward_queue, forward_visited, backward_visited)
        if meeting_point:
            return construct_path(meeting_point, forward_visited, backward_visited)

        # Backward search step
        meeting_point = expand_frontier(graph, backward_queue, backward_visited, forward_visited)
        if meeting_point:
            return construct_path(meeting_point, forward_visited, backward_visited)

    return None  # No path found

def expand_frontier(graph, queue, visited, other_visited):
    current = queue.popleft()
    for neighbor in graph[current]:
        if neighbor not in visited:
            visited[neighbor] = current
            queue.append(neighbor)

            # Check if this neighbor is visited by the other search
            if neighbor in other_visited:
                return neighbor
    return None

def construct_path(meeting_point, forward_visited, backward_visited):
    # Construct path from start to meeting point
    path = []
    node = meeting_point
    while node:
        path.append(node)
        node = forward_visited[node]
    path.reverse()  # Reverse to get correct order from start to meeting point

    # Construct path from meeting point to goal
    node = backward_visited[meeting_point]
    while node:
        path.append(node)
        node = backward_visited[node]

    return path

# Example graph represented as an adjacency list
graph = {
    'A': ['B', 'C'],
    'B': ['A', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['B'],
    'E': ['B', 'F'],
    'F': ['C', 'E']
}

# Find the shortest path from A to F
path = bidirectional_search(graph, 'A', 'F')
if path:
    print("Shortest Path:", " -> ".join(path))
else:
    print("No path found between the nodes.")




Shortest Path: A -> C -> F


In [8]:
import itertools

# Distance matrix (example with 4 cities: A, B, C, D)
# 0 represents no self-loop distance
distance_matrix = [
    [0, 10, 15, 20],   # Distances from A
    [10, 0, 35, 25],   # Distances from B
    [15, 35, 0, 30],   # Distances from C
    [20, 25, 30, 0]    # Distances from D
]

# List of cities (represented by indices)
cities = [0, 1, 2, 3]  # 0:A, 1:B, 2:C, 3:D

def calculate_total_distance(route, distance_matrix):
    total_distance = 0
    # Calculate distance from city to city in route
    for i in range(len(route) - 1):
        total_distance += distance_matrix[route[i]][route[i+1]]
    # Add distance to return to the starting city
    total_distance += distance_matrix[route[-1]][route[0]]
    return total_distance

def tsp_brute_force(distance_matrix):
    # Starting from the first city (0)
    start_city = cities[0]

    # Generate all permutations of the remaining cities
    min_distance = float('inf')
    optimal_route = []

    for perm in itertools.permutations(cities[1:]):  # Exclude the starting city
        # Create the full route starting and ending at start_city
        current_route = [start_city] + list(perm)
        total_distance = calculate_total_distance(current_route, distance_matrix)

        # Update if a shorter route is found
        if total_distance < min_distance:
            min_distance = total_distance
            optimal_route = current_route

    # Convert city indices back to labels (A, B, C, D)
    city_labels = ['A', 'B', 'C', 'D']
    optimal_route_labels = [city_labels[city] for city in optimal_route]

    print("Optimal Route:", " -> ".join(optimal_route_labels + [optimal_route_labels[0]]))  # Returning to start
    print("Minimum Distance:", min_distance)

# Run the brute force TSP
tsp_brute_force(distance_matrix)


Optimal Route: A -> B -> D -> C -> A
Minimum Distance: 80
